### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
from time import time
from tqdm import tqdm
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [17]:
np.random.rand()

0.29718440668560075

In [3]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]  # number of rows in the matrix
    n_movies = ratings_mat.shape[1]  # number of movies in the matrix
    num_ratings = (1 - np.isnan(ratings_mat)).sum()  # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    min_rating, max_rating = (1, 10)
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    mean_iter_time = 0
    for i in range(iters):
        tic = time()
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for u in tqdm(range(n_users)):
            for m in range(n_movies):
                if not np.isnan(ratings_mat[u, m]):
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = ratings_mat[u, m] - user_mat[u, :].dot(movie_mat[:, m])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error**2
                    
                    # update the values in each matrix in the direction of the gradient
                    user_mat[u, :] += learning_rate * 2 * error * movie_mat[:, m]
                    movie_mat[:, m] += learning_rate * 2 * error * user_mat[u, :]
        toc = time()
                    
        # print results for iteration
        iter_time = toc - tic
        mean_iter_time = (mean_iter_time * i + iter_time) / (i + 1)
        eta = mean_iter_time * (iters - i -1)
        print('Iteration {} | {} -- Iter time: {} sec. ETA: {} sec'.format(
            i, sse_accum / num_ratings, iter_time, eta))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [4]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)
# use your function with 4 latent features, lr of 0.005 and 10 iterations

100%|██████████| 20/20 [00:00<00:00, 2169.45it/s]

Optimization Statistics
Iterations | Mean Squared Error 
Iteration 0 | 48.758815098045 -- Iter time: 0.009335756301879883 sec. ETA: 0.08402180671691895 sec
Iteration 1 | 19.352768342810617 -- Iter time: 0.0077550411224365234 sec. ETA: 0.06836318969726562 sec
Iteration 2 | 4.912137351014641 -- Iter time: 0.006158113479614258 sec. ETA: 0.05424745877583821 sec
Iteration 3 | 2.899248418635939 -- Iter time: 0.005427122116088867 sec. ETA: 0.0430140495300293 sec
Iteration 4 | 2.7569771186235426 -- Iter time: 0.014972925186157227 sec. ETA: 0.04364895820617676 sec
Iteration 5 | 2.72106994979535 -- Iter time: 0.008913040161132812 sec. ETA: 0.03504133224487305 sec
Iteration 6 | 2.6916651440961648 -- Iter time: 0.008634805679321289 sec. ETA: 0.026227201734270372 sec
Iteration 7 | 2.660425660873849 -- Iter time: 0.009706974029541016 sec. ETA: 0.01772594451904297 sec
Iteration 8 | 2.625481697029229 -- Iter time: 0.012928009033203125 sec. ETA: 0.009314643012152778 sec
Iteration 9 | 2.5859319854110585

In [5]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.17284711  9.04758688 10.66385153  9.94908877]
 [ 8.62258584  7.43233012  9.31515886  8.63226287]
 [ 7.6210927   6.84469039  8.50138972  7.68117199]
 [ 9.58744304  8.4013951   9.74227532  9.75376647]
 [ 8.67042162  7.58658114  8.7811012   8.54604007]
 [ 6.69634201  5.89278987  7.27924709  6.94380275]
 [ 9.3267497   8.08896863  9.50020664  9.26457908]
 [ 8.13410188  7.16235601  8.68736861  8.17456484]
 [ 8.16971683  7.15742566  9.29591987  8.47200849]
 [ 7.62760315  6.62449824  8.15555185  7.67623551]
 [ 9.18227751  7.99795966  8.8807156   8.95002686]
 [ 9.88615321  8.423767    9.80336953  9.61665618]
 [ 9.10873188  8.07048889  9.76807141  9.3144106 ]
 [ 6.70097932  5.9253199   6.79101468  6.65861496]
 [ 9.7554547   8.55343306 10.24267044  9.7069661 ]
 [ 9.74907532  8.4314104  10.19551076  9.81829753]
 [ 8.33953824  7.27647746  8.7473638   8.20384759]
 [ 7.44870141  6.30987798  6.94095934  7.26948433]
 [ 8.35688065  7.28373021  8.89231803  8.38621354]
 [ 7.87848067  7.00095116  8.82

The algorithm seems to converge and the predicted matrix is similar to the acutal one. In any case the matrix under test has no missing values, so this was an easy case.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [6]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, 
                              learning_rate=0.005, iters=250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

100%|██████████| 20/20 [00:00<00:00, 2853.85it/s]

Optimization Statistics
Iterations | Mean Squared Error 
Iteration 0 | 44.001898704518446 -- Iter time: 0.008318901062011719 sec. ETA: 2.071406364440918 sec
Iteration 1 | 15.131324867551902 -- Iter time: 0.008056163787841797 sec. ETA: 2.030508041381836 sec
Iteration 2 | 4.115179205576126 -- Iter time: 0.005526065826416016 sec. ETA: 1.8031930923461914 sec
Iteration 3 | 2.8002822170305346 -- Iter time: 0.010019063949584961 sec. ETA: 1.9630919694900513 sec
Iteration 4 | 2.693577996708192 -- Iter time: 0.006443023681640625 sec. ETA: 1.8797976970672607 sec
Iteration 5 | 2.6713434271323253 -- Iter time: 0.013384342193603516 sec. ETA: 2.1044007937113443 sec
Iteration 6 | 2.658877702514063 -- Iter time: 0.009544134140014648 sec. ETA: 2.127697399684361 sec
Iteration 7 | 2.646976996608253 -- Iter time: 0.0057032108306884766 sec. ETA: 2.026595890522003 sec
Iteration 8 | 2.6339690981264194 -- Iter time: 0.00554203987121582 sec. ETA: 1.9423782030741372 sec
Iteration 9 | 2.6196065063578304 -- Iter t


100%|██████████| 20/20 [00:00<00:00, 4356.13it/s]


Iteration 20 | 2.361789099768556 -- Iter time: 0.017410755157470703 sec. ETA: 2.36322393871489 sec
Iteration 21 | 2.3291565342346168 -- Iter time: 0.01517176628112793 sec. ETA: 2.4031886620955034 sec
Iteration 22 | 2.295196884910007 -- Iter time: 0.008558034896850586 sec. ETA: 2.373084254886793 sec
Iteration 23 | 2.25997217053878 -- Iter time: 0.015062093734741211 sec. ETA: 2.4060219327608743 sec
Iteration 24 | 2.223526226737551 -- Iter time: 0.014061212539672852 sec. ETA: 2.4261116981506348 sec
Iteration 25 | 2.185877075044006 -- Iter time: 0.013810873031616211 sec. ETA: 2.441417694091797 sec
Iteration 26 | 2.147011412298075 -- Iter time: 0.010787725448608398 sec. ETA: 2.4295979252567994 sec
Iteration 27 | 2.106882175553292 -- Iter time: 0.007863044738769531 sec. ETA: 2.3946633338928223 sec
Iteration 28 | 2.065409909211117 -- Iter time: 0.01318502426147461 sec. ETA: 2.4021529000380943 sec
Iteration 29 | 2.0224883148366506 -- Iter time: 0.010406017303466797 sec. ETA: 2.387884775797526 

  0%|          | 0/20 [00:00<?, ?it/s]

...

Iteration 187 | 4.032690507525349e-08 -- Iter time: 0.010725021362304688 sec. ETA: 0.6935459096381004 sec
Iteration 188 | 3.582552502723672e-08 -- Iter time: 0.01285409927368164 sec. ETA: 0.682897994127223 sec
Iteration 189 | 3.182625092004343e-08 -- Iter time: 0.010415792465209961 sec. ETA: 0.6714568640056408 sec
Iteration 190 | 2.8273117106685717e-08 -- Iter time: 0.009079933166503906 sec. ETA: 0.6596138227672476 sec
Iteration 191 | 2.511639109955619e-08 -- Iter time: 0.01576709747314453 sec. ETA: 0.6498196447889009 sec
Iteration 192 | 2.231187990342604e-08 -- Iter time: 0.008021116256713867 sec. ETA: 0.6376758980627504 sec
Iteration 193 | 1.98203134865228e-08 -- Iter time: 0.010740280151367188 sec. ETA: 0.6263595659708238 sec
Iteration 194 | 1.7606796815701726e-08 -- Iter time: 0.01092386245727539 sec. ETA: 0.615100921728672 sec
Iteration 195 | 1.564032283684956e-08 -- Iter time: 0.0072062015533447266 sec. ETA: 0.602821440112834 sec
Iteration 196 | 1.3893339626837598e-08 -- Iter tim

  0%|          | 0/20 [00:00<?, ?it/s]

Iteration 207 | 3.773118027100286e-09 -- Iter time: 0.00981903076171875 sec. ETA: 0.46585327616104705 sec
Iteration 208 | 3.351293526438838e-09 -- Iter time: 0.009765148162841797 sec. ETA: 0.4545012900703831 sec
Iteration 209 | 2.976602439944208e-09 -- Iter time: 0.009569168090820312 sec. ETA: 0.44312708718436095 sec
Iteration 210 | 2.643781407065511e-09 -- Iter time: 0.008109092712402344 sec. ETA: 0.4315001218804815 sec
Iteration 211 | 2.3481544285441366e-09 -- Iter time: 0.01131892204284668 sec. ETA: 0.4204816908206579 sec
Iteration 212 | 2.0855673687325607e-09 -- Iter time: 0.009701013565063477 sec. ETA: 0.4091793931146182 sec
Iteration 213 | 1.8523297554379853e-09 -- Iter time: 0.010344982147216797 sec. ETA: 0.398000391844277 sec
Iteration 214 | 1.6451630656917205e-09 -- Iter time: 0.013011932373046875 sec. ETA: 0.387263303579286 sec
Iteration 215 | 1.4611547747165388e-09 -- Iter time: 0.009924173355102539 sec. ETA: 0.3760191158012107 sec
Iteration 216 | 1.2977175266033585e-09 -- I

100%|██████████| 20/20 [00:00<00:00, 6029.33it/s]

Iteration 228 | 3.124265815608036e-10 -- Iter time: 0.012666940689086914 sec. ETA: 0.23017825726338345 sec
Iteration 229 | 2.774556537916606e-10 -- Iter time: 0.005366086959838867 sec. ETA: 0.21873088504957114 sec
Iteration 230 | 2.463976653140129e-10 -- Iter time: 0.012033939361572266 sec. ETA: 0.20788460273247258 sec
Iteration 231 | 2.1881498183636058e-10 -- Iter time: 0.011111974716186523 sec. ETA: 0.19695655025284864 sec
Iteration 232 | 1.9431889118460013e-10 -- Iter time: 0.007990121841430664 sec. ETA: 0.1857991443683149 sec
Iteration 233 | 1.7256413741891133e-10 -- Iter time: 0.009276151657104492 sec. ETA: 0.1747567429501786 sec
Iteration 234 | 1.5324406538866293e-10 -- Iter time: 0.009078025817871094 sec. ETA: 0.16371672711473828 sec
Iteration 235 | 1.3608630731923835e-10 -- Iter time: 0.010142087936401367 sec. ETA: 0.1527564606424105 sec
Iteration 236 | 1.2084895115060888e-10 -- Iter time: 0.008240938186645508 sec. ETA: 0.14169881615457655 sec
Iteration 237 | 1.0731713687481774

In [7]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999357  9.99999673  9.99999802 10.0000069 ]
 [ 9.99999696  3.99999852  8.99999914 10.0000033 ]
 [ 8.00000036  9.00000017 10.00000016  4.99999976]
 [ 8.99999536  7.99999783  9.99999874 10.00000497]
 [10.0000042   5.00000214  9.00000139  8.9999957 ]
 [ 6.00000392  4.00000188 10.00000119  5.99999599]
 [ 8.99999736  7.99999867  9.99999925  9.00000291]
 [ 9.99998814  4.99999421  8.99999654  8.00001261]
 [ 6.99999319  7.99999688  9.99999821  8.00000727]
 [ 9.00000494  5.00000273  9.00000177  6.99999487]
 [ 9.00000617  8.00000324 10.00000208  7.99999361]
 [ 9.00000216 10.00000116 10.0000008   8.99999784]
 [ 9.99999028  8.99999532  9.99999722  8.00001036]
 [ 5.00000168  8.00000108  5.00000075  7.99999824]
 [10.00001115  8.00000566 10.00000355  9.99998836]
 [ 9.00000007  9.00000005 10.00000005 10.        ]
 [ 9.00000537  8.00000261  8.00000161  7.99999443]
 [10.00000024  8.00000007  1.00000005  9.99999979]
 [ 4.99999893  5.99999939  9.99999962 10.00000118]
 [ 8.          6.99999991  9.99

The predicted and the actual matrices are basically the same. The error after 250 iterations is zero, for all practical means.

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [8]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [9]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, 
                              learning_rate=0.005, iters=250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

100%|██████████| 20/20 [00:00<00:00, 6420.67it/s]

Optimization Statistics
Iterations | Mean Squared Error 
Iteration 0 | 42.020098491256505 -- Iter time: 0.0065310001373291016 sec. ETA: 1.6262190341949463 sec
Iteration 1 | 14.90571501980531 -- Iter time: 0.007200002670288086 sec. ETA: 1.7026443481445312 sec
Iteration 2 | 4.288855832498844 -- Iter time: 0.0045680999755859375 sec. ETA: 1.5066261291503906 sec
Iteration 3 | 2.8754430890846017 -- Iter time: 0.006797075271606445 sec. ETA: 1.5434149503707886 sec
Iteration 4 | 2.741064689064605 -- Iter time: 0.010975122451782227 sec. ETA: 1.767493724822998 sec
Iteration 5 | 2.7075536144918786 -- Iter time: 0.016086101531982422 sec. ETA: 2.121067682902018 sec
Iteration 6 | 2.6836563141320875 -- Iter time: 0.006746053695678711 sec. ETA: 2.0447913919176375 sec



  0%|          | 0/20 [00:00<?, ?it/s]

Iteration 7 | 2.6594368763728715 -- Iter time: 0.009907960891723633 sec. ETA: 2.081545352935791 sec
Iteration 8 | 2.6330518998702033 -- Iter time: 0.01913309097290039 sec. ETA: 2.3549584812588162 sec
Iteration 9 | 2.6038482776858447 -- Iter time: 0.012302160263061523 sec. ETA: 2.4059200286865234 sec
Iteration 10 | 2.5713466847732 -- Iter time: 0.014060020446777344 sec. ETA: 2.4835725914348252 sec
Iteration 11 | 2.535102121960849 -- Iter time: 0.007738828659057617 sec. ETA: 2.42056941986084 sec
Iteration 12 | 2.4947165454229667 -- Iter time: 0.013025999069213867 sec. ETA: 2.462457638520461 sec
Iteration 13 | 2.449876432890763 -- Iter time: 0.015049934387207031 sec. ETA: 2.530618735722133 sec
Iteration 14 | 2.400394429295841 -- Iter time: 0.00896596908569336 sec. ETA: 2.4923695723215737 sec
Iteration 15 | 2.346250951988085 -- Iter time: 0.012180805206298828 sec. ETA: 2.504797786474228 sec
Iteration 16 | 2.2876317471676506 -- Iter time: 0.016199827194213867 sec. ETA: 2.569415064419017 sec

100%|██████████| 20/20 [00:00<00:00, 4859.30it/s]

...

Iteration 216 | 1.464720769439912e-10 -- Iter time: 0.011405229568481445 sec. ETA: 0.3552149748472573 sec
Iteration 217 | 1.2994827208367652e-10 -- Iter time: 0.009839773178100586 sec. ETA: 0.34431520514531955 sec
Iteration 218 | 1.1528924578935117e-10 -- Iter time: 0.009954214096069336 sec. ETA: 0.33344131517628 sec
Iteration 219 | 1.0228439758162068e-10 -- Iter time: 0.008347272872924805 sec. ETA: 0.3223566575483841 sec
Iteration 220 | 9.074693968560952e-11 -- Iter time: 0.010317802429199219 sec. ETA: 0.3115553489098181 sec
Iteration 221 | 8.051119965646584e-11 -- Iter time: 0.006088972091674805 sec. ETA: 0.30022503019453156 sec
Iteration 222 | 7.143022937598952e-11 -- Iter time: 0.012878894805908203 sec. ETA: 0.28976381733813084 sec
Iteration 223 | 6.337368542899504e-11 -- Iter time: 0.010308980941772461 sec. ETA: 0.2789827244622366 sec
Iteration 224 | 5.622594997660082e-11 -- Iter time: 0.012918949127197266 sec. ETA: 0.26849582460191507 sec
Iteration 225 | 4.9884464787252254e-11 --

100%|██████████| 20/20 [00:00<00:00, 4316.02it/s]

Iteration 231 | 2.4329791348312137e-11 -- Iter time: 0.013098001480102539 sec. ETA: 0.19289416896885833 sec
Iteration 232 | 2.158567936717402e-11 -- Iter time: 0.011469125747680664 sec. ETA: 0.18223275033189495 sec
Iteration 233 | 1.9151030930564075e-11 -- Iter time: 0.011012077331542969 sec. ETA: 0.17153317704159984 sec
Iteration 234 | 1.699094502453869e-11 -- Iter time: 0.009922027587890625 sec. ETA: 0.16076136649923115 sec
Iteration 235 | 1.507445788126229e-11 -- Iter time: 0.006823062896728516 sec. ETA: 0.1498129206188654 sec



100%|██████████| 20/20 [00:00<00:00, 7079.59it/s]

Iteration 236 | 1.3374098559521004e-11 -- Iter time: 0.010473012924194336 sec. ETA: 0.139099496326366 sec
Iteration 237 | 1.1865494692665207e-11 -- Iter time: 0.007253170013427734 sec. ETA: 0.1282257472767549 sec
Iteration 238 | 1.0527022824148942e-11 -- Iter time: 0.009229183197021484 sec. ETA: 0.117473242173135 sec
Iteration 239 | 9.339498208825833e-12 -- Iter time: 0.012717008590698242 sec. ETA: 0.1068787574768066 sec
Iteration 240 | 8.285899665768915e-12 -- Iter time: 0.007941007614135742 sec. ETA: 0.0960883015913587 sec
Iteration 241 | 7.351125485154125e-12 -- Iter time: 0.01112508773803711 sec. ETA: 0.08542665371224897 sec
Iteration 242 | 6.521776907722692e-12 -- Iter time: 0.009529829025268555 sec. ETA: 0.07471523755862386 sec
Iteration 243 | 5.7859660351408526e-12 -- Iter time: 0.00862884521484375 sec. ETA: 0.06399135120579452 sec
Iteration 244 | 5.133145423824163e-12 -- Iter time: 0.005609035491943359 sec. ETA: 0.053222938459746676 sec
Iteration 245 | 4.553956908937936e-12 -- 

In [10]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.328867452974876:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [11]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, 
                              learning_rate=0.005, iters=20)
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

  0%|          | 2/1000 [00:00<01:28, 11.33it/s]

Optimization Statistics
Iterations | Mean Squared Error 


  0%|          | 2/1000 [00:00<01:27, 11.46it/s]

Iteration 0 | 0.008045903269022613 -- Iter time: 85.91714286804199 sec. ETA: 1632.4257144927979 sec


  0%|          | 2/1000 [00:00<01:31, 10.92it/s]

...

Iteration 17 | 0.00042638783679579473 -- Iter time: 86.44891905784607 sec. ETA: 176.4839939541287 sec


  0%|          | 1/1000 [00:00<01:51,  8.96it/s]

Iteration 18 | 0.0004030255592360763 -- Iter time: 87.84607791900635 sec. ETA: 88.22115913190339 sec


100%|██████████| 1000/1000 [01:24<00:00, 11.88it/s]

Iteration 19 | 0.0003823023166040969 -- Iter time: 84.17219424247742 sec. ETA: 0.0 sec


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [25]:
preds = np.dot(user_mat, movie_mat)

In [26]:
np.isnan(preds).sum()

0

In [24]:
(1 - np.isnan(preds)).sum()

31245000

In [27]:
# Replace each of the comments below with the correct values
num_ratings = (1 - np.isnan(first_1000_users)).sum()
# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = (1 - np.isnan(preds)).sum() - num_ratings
# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [28]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
